<a href="https://colab.research.google.com/github/RajanTucker/RajanTucker/blob/main/Spam_Ham__Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [74]:
import numpy as np # for array
import pandas as pd # for data frame to structure data
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
import nltk


Reading Data file ie maildata.csv

In [90]:
cluster_df=pd.read_csv('/content/Email Data/clusterdata.csv')

In [91]:
cluster_df

,text,spam
0,Subject: naturally irresistible your corporate...,1.0
1,Subject: the stock trading gunslinger fanny i...,1.0
2,Subject: unbelievable new homes made easy im ...,1.0
3,Subject: 4 color printing special request add...,1.0
4,"Subject: do not have money , get software cds ...",1.0
...,...,...
5731,Subject: re : research and development charges...,0.0
5732,"Subject: re : receipts from visit jim , than...",0.0
5733,Subject: re : enron case study update wow ! a...,0.0
5734,"Subject: re : interest david , please , call...",0.0


In [92]:
cluster_df.head()


,text,spam
0,Subject: naturally irresistible your corporate...,1.0
1,Subject: the stock trading gunslinger fanny i...,1.0
2,Subject: unbelievable new homes made easy im ...,1.0
3,Subject: 4 color printing special request add...,1.0
4,"Subject: do not have money , get software cds ...",1.0


In [93]:
cluster_df.tail()

,text,spam
5731,Subject: re : research and development charges...,0.0
5732,"Subject: re : receipts from visit jim , than...",0.0
5733,Subject: re : enron case study update wow ! a...,0.0
5734,"Subject: re : interest david , please , call...",0.0
5735,Subject: news : aurora 5 . 2 update aurora ve...,0.0


In [94]:
cluster_df.shape

(5736, 2)

In [95]:

cluster_df= cluster_df.where((pd.notnull(cluster_df)),"")

In [96]:
cluster_df.shape


(5736, 2)

In [97]:
print("no of rows",len(cluster_df))

no of rows 5736


In [98]:
cluster_df.spam.value_counts()

0.0    4360
1.0    1368
          8
Name: spam, dtype: int64

In [99]:
cluster_df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1.0
1,Subject: the stock trading gunslinger fanny i...,1.0
2,Subject: unbelievable new homes made easy im ...,1.0
3,Subject: 4 color printing special request add...,1.0
4,"Subject: do not have money , get software cds ...",1.0


### I want to change columns text heading to Subject as text sounds more of phone text **messages**

In [100]:
cluster_df.columns=["Subject","Spam"]

In [101]:
cluster_df.head()

,Subject,Spam
0,Subject: naturally irresistible your corporate...,1.0
1,Subject: the stock trading gunslinger fanny i...,1.0
2,Subject: unbelievable new homes made easy im ...,1.0
3,Subject: 4 color printing special request add...,1.0
4,"Subject: do not have money , get software cds ...",1.0


In [102]:
cluster_df.Spam.value_counts()

0.0    4360
1.0    1368
          8
Name: Spam, dtype: int64

In [103]:
cluster_df.describe()

,Subject,Spam
count,5736,5736.0
unique,5698,3.0
top,,0.0
freq,6,4360.0
